In [1]:
import pymongo
import pandas as pd
import numpy as np
import datetime

In [2]:
client = pymongo.MongoClient()
db = client['Tick']

In [34]:
# 讀取連續資料做回測
stk_test = pd.DataFrame(db['STK'].find({}, {'_id': 0}))
stk_test['datetime'] = pd.to_datetime(stk_test['datetime'])
#stk_return = stk_test[(stk_test['datetime'] >= datetime.datdatetime.datetime.today(); start.minute=0; start.hour=9; start.second=0etime(2022,10,21,9,0,0)) & (stk_test['datetime'] < datetime.datetime(2022,10,21,13,25,0))]
#stk_return = stk_test[stk_test['datetime'] < datetime.datetime(2022,10,21,13,25,0)]
stk_test = stk_test[stk_test['datetime'] > datetime.datetime(2022,10,25,9,0,0)]
stk_test = stk_test[stk_test['datetime'] < datetime.datetime(2022,10,25,13,25,0)]
stk_test.set_index('datetime', inplace=True)


In [35]:
total_list = list()
for code in stk_test['code'].unique():
    time_list = list()
    single_stk_list = list()
    for t in stk_test[stk_test['code'] == code].index:
        time_package = [t - datetime.timedelta(milliseconds=0.05), t + datetime.timedelta(milliseconds=0.05)]
        time_list.append(t)
        single_stk_list.append(stk_test[(stk_test.index > time_package[0]) & (stk_test.index <= time_package[1])].groupby('datetime').sum())

    # 左側外盤成交(買), 右側內盤成交(賣)
    total = [0, 0]
    iso = [0, 0]
    nis = [0, 0]
    nis_s = [0, 0]
    nis_c = [0, 0]
    nis_b = [0, 0]

    for i in range(len(single_stk_list)):
        tick_type = single_stk_list[i].at[time_list[i], 'tick_type']
        volume = int(single_stk_list[i].at[time_list[i], 'volume'])
        data = single_stk_list[i]
        if len(data) == 1:
            if tick_type == '1': # 外盤成交
                iso[0] += volume
            elif tick_type == '2': # 內盤成交
                iso[1] += volume
        else:
            if len(data['code'].unique()) == 1:
                if tick_type == '1': # 外盤成交
                    nis_s[0] += volume
                elif tick_type == '2': # 內盤成交
                    nis_s[1] += volume
            else:
                if len(data[data['code'] == '2609']) == 1:
                    if tick_type == '1': # 外盤成交
                        nis_c[0] += volume
                    elif tick_type == '2': # 內盤成交
                        nis_c[1] += volume
                else:
                    if tick_type == '1': # 外盤成交
                        nis_b[0] += volume
                    elif tick_type == '2': # 內盤成交
                        nis_b[1] += volume
    nis = np.sum([nis_s, nis_c, nis_b], axis=0).tolist()
    total = np.sum([iso, nis], axis=0).tolist()
    total_list.append(pd.DataFrame(dict(zip([(code, 'Long'), (code, 'Short')], zip(total, iso, nis, nis_s, nis_c, nis_b)))))
    print('code:{} total {}, iso {}, nis {}, nis_s {}, nis_c {}, nis_b {}'.format(code, total, iso, nis, nis_s, nis_c, nis_b))




code:3037 total [17814, 20080], iso [17814, 20080], nis [0, 0], nis_s [0, 0], nis_c [0, 0], nis_b [0, 0]
code:2303 total [36262, 38630], iso [36262, 38630], nis [0, 0], nis_s [0, 0], nis_c [0, 0], nis_b [0, 0]
code:3552 total [455, 933], iso [455, 933], nis [0, 0], nis_s [0, 0], nis_c [0, 0], nis_b [0, 0]
code:2330 total [21171, 38767], iso [21171, 38767], nis [0, 0], nis_s [0, 0], nis_c [0, 0], nis_b [0, 0]
code:3443 total [4552, 4168], iso [4552, 4168], nis [0, 0], nis_s [0, 0], nis_c [0, 0], nis_b [0, 0]
code:3189 total [10570, 10772], iso [10570, 10772], nis [0, 0], nis_s [0, 0], nis_c [0, 0], nis_b [0, 0]
code:8046 total [7116, 7160], iso [7116, 7160], nis [0, 0], nis_s [0, 0], nis_c [0, 0], nis_b [0, 0]
code:2609 total [14790, 14062], iso [14790, 14062], nis [0, 0], nis_s [0, 0], nis_c [0, 0], nis_b [0, 0]
code:2498 total [5552, 7813], iso [5552, 7813], nis [0, 0], nis_s [0, 0], nis_c [0, 0], nis_b [0, 0]
code:2603 total [14116, 11403], iso [14116, 11403], nis [0, 0], nis_s [0, 0

In [36]:
total_df = pd.concat(total_list, axis=1)
total_df.index = ['all', 'iso', 'nis', 'nis_s', 'nis_c', 'nis_b']
total_df.groupby(level=[0]).sum()

3037          2303        3552         2330         3443        ...  \
        Long  Short   Long  Short Long Short   Long  Short  Long Short  ...   
all    17814  20080  36262  38630  455   933  21171  38767  4552  4168  ...   
iso    17814  20080  36262  38630  455   933  21171  38767  4552  4168  ...   
nis        0      0      0      0    0     0      0      0     0     0  ...   
nis_b      0      0      0      0    0     0      0      0     0     0  ...   
nis_c      0      0      0      0    0     0      0      0     0     0  ...   
nis_s      0      0      0      0    0     0      0      0     0     0  ...   

       9945        2888         5425        6285        2474        
       Long Short  Long  Short  Long Short  Long Short  Long Short  
all    3550  3122  6895  10258  4985  7007  3295  3247  1033   875  
iso    3550  3122  6895  10258  4985  7007  3295  3247  1033   875  
nis       0     0     0      0     0     0     0     0     0     0  
nis_b     0     0     0      0     0     0     0     0     0     0  
nis_c     0     0     0      0     0     0     0     0     0     0  
nis_s     0     0     0      0     0     0     0     0     0     0  

[6 rows x 98 columns]

In [37]:
# 計算(COI_b - COI_s) / (COI_b + COI_s)
# total_df.groupby(level=0, axis=1).diff(-1).dropna(axis=1).droplevel(level=1, axis=1)
score = total_df.groupby(level=0, axis=1).diff(-1).dropna(axis=1).droplevel(level=1, axis=1) / \
    total_df.groupby(level=[0], axis=1).sum()
score.fillna(0, inplace=True)
ls = dict()
rank = score.rank(axis=1)
for x in rank.index:
    ls[(x, 'long')] = rank.loc[x, :].sort_values()[:5].index
    ls[(x, 'short')] = rank.loc[x, :].sort_values()[-5:].index
    for y in rank.index:
        if x == y:
            continue
        ls[(x + '-' + y, 'long')] = (rank.loc[x, :] - rank.loc[y, :]).sort_values()[:5].index
        ls[(x + '-' + y, 'short')] = (rank.loc[x, :] - rank.loc[y, :]).sort_values()[-5:].index
(("=@XQ|Quote!'"+ pd.DataFrame(ls) +".TW-Price'") + '/' + ("@XQ|Quote!'"+ pd.DataFrame(ls) +".TW-Open'") + '-1').to_excel(r'C:\Users\ChastLai\Documents\Python Scripts\Output\PaperTrade\COI\COI.xlsx')


In [3]:
# 讀取連續資料做回測
from datetime import timedelta

stk_all = pd.DataFrame(db['STK'].find({}, {'_id': 0}))
stk_all['datetime'] = pd.to_datetime(stk_all['datetime'])
#stk_all.set_index('datetime', inplace=True)
total_data = dict()
start = datetime.datetime(2022, 10, 21, 9, 0, 0)
for _ in range(20):
    try:
        start = start + timedelta(minutes=15)
        end = start + timedelta(minutes=15)       
        #stk_return = stk_test[(stk_test['datetime'] >= datetime.datetime(2022,10,21,9,0,0)) & (stk_test['datetime'] < datetime.datetime(2022,10,21,13,25,0))]
        #stk_return = stk_test[stk_test['datetime'] < datetime.datetime(2022,10,21,13,25,0)]
        stk_test = stk_all.loc[stk_all['datetime'] > start]
        stk_test = stk_test[stk_test['datetime'] < end]
        stk_test.set_index('datetime', inplace=True)
        total_list = list()
        for code in stk_test['code'].unique():
            time_list = list()
            single_stk_list = list()
            for t in stk_test[stk_test['code'] == code].index:
                time_package = [t - datetime.timedelta(milliseconds=0.05), t + datetime.timedelta(milliseconds=0.05)]
                time_list.append(t)
                single_stk_list.append(stk_test[(stk_test.index > time_package[0]) & (stk_test.index <= time_package[1])].groupby('datetime').sum())

            # 左側外盤成交(買), 右側內盤成交(賣)
            total = [0, 0]
            iso = [0, 0]
            nis = [0, 0]
            nis_s = [0, 0]
            nis_c = [0, 0]
            nis_b = [0, 0]

            for i in range(len(single_stk_list)):
                tick_type = single_stk_list[i].at[time_list[i], 'tick_type']
                volume = int(single_stk_list[i].at[time_list[i], 'volume'])
                data = single_stk_list[i]
                if len(data) == 1:
                    if tick_type == '1': # 外盤成交
                        iso[0] += volume
                    elif tick_type == '2': # 內盤成交
                        iso[1] += volume
                else:
                    if len(data['code'].unique()) == 1:
                        if tick_type == '1': # 外盤成交
                            nis_s[0] += volume
                        elif tick_type == '2': # 內盤成交
                            nis_s[1] += volume
                    else:
                        if len(data[data['code'] == '2609']) == 1:
                            if tick_type == '1': # 外盤成交
                                nis_c[0] += volume
                            elif tick_type == '2': # 內盤成交
                                nis_c[1] += volume
                        else:
                            if tick_type == '1': # 外盤成交
                                nis_b[0] += volume
                            elif tick_type == '2': # 內盤成交
                                nis_b[1] += volume
            nis = np.sum([nis_s, nis_c, nis_b], axis=0).tolist()
            total = np.sum([iso, nis], axis=0).tolist()
            total_list.append(pd.DataFrame(dict(zip([(code, 'Long'), (code, 'Short')], zip(total, iso, nis, nis_s, nis_c, nis_b)))))
            print('code:{} total {}, iso {}, nis {}, nis_s {}, nis_c {}, nis_b {}'.format(code, total, iso, nis, nis_s, nis_c, nis_b))

        total_df = pd.concat(total_list, axis=1)
        total_df.index = ['all', 'iso', 'nis', 'nis_s', 'nis_c', 'nis_b']
        total_data[str(start)+'-'+str(end)] = total_df.groupby(level=[0]).sum()
    except Exception as e:
        print(e)

code:2409 total [3637, 3746], iso [3635, 3695], nis [2, 51], nis_s [2, 50], nis_c [0, 0], nis_b [0, 1]
code:2618 total [2576, 5136], iso [2575, 5060], nis [1, 76], nis_s [0, 67], nis_c [0, 0], nis_b [1, 9]
code:2888 total [818, 1373], iso [818, 1333], nis [0, 40], nis_s [0, 35], nis_c [0, 0], nis_b [0, 5]
code:5425 total [1080, 667], iso [1064, 667], nis [16, 0], nis_s [16, 0], nis_c [0, 0], nis_b [0, 0]
code:2303 total [4140, 5082], iso [3876, 5053], nis [264, 29], nis_s [260, 3], nis_c [0, 0], nis_b [4, 26]
code:2615 total [298, 663], iso [298, 638], nis [0, 25], nis_s [0, 23], nis_c [0, 0], nis_b [0, 2]
code:3035 total [1613, 1581], iso [1610, 1555], nis [3, 26], nis_s [2, 26], nis_c [0, 0], nis_b [1, 0]
code:6285 total [206, 954], iso [206, 943], nis [0, 11], nis_s [0, 8], nis_c [0, 0], nis_b [0, 3]
code:3406 total [159, 343], iso [156, 340], nis [3, 3], nis_s [0, 3], nis_c [1, 0], nis_b [2, 0]
code:3037 total [3086, 2192], iso [3050, 2079], nis [36, 113], nis_s [31, 111], nis_c [0

In [5]:
from IPython.display import display
for k, v in total_data.items():
    display(k)
    display(v.sum(axis=1))
    

'2022-10-21 09:15:00-2022-10-21 09:30:00'

all      111396
iso      108143
nis        3253
nis_b       233
nis_c         2
nis_s      3018
dtype: int64

'2022-10-21 09:30:00-2022-10-21 09:45:00'

all      65286
iso      64500
nis        786
nis_b      101
nis_c        0
nis_s      685
dtype: int64

'2022-10-21 09:45:00-2022-10-21 10:00:00'

...

all      32377
iso      31829
nis        548
nis_b      195
nis_c        0
nis_s      353
dtype: int64

'2022-10-21 13:00:00-2022-10-21 13:15:00'

all      51360
iso      49864
nis       1496
nis_b      295
nis_c        3
nis_s     1198
dtype: int64

'2022-10-21 13:15:00-2022-10-21 13:30:00'

all      1531113
iso      1530596
nis          517
nis_b        276
nis_c          9
nis_s        232
dtype: int64

In [6]:
# 計算(COI_b - COI_s) / (COI_b + COI_s)
# total_df.groupby(level=0, axis=1).diff(-1).dropna(axis=1).droplevel(level=1, axis=1)
score = total_df.groupby(level=0, axis=1).diff(-1).dropna(axis=1).droplevel(level=1, axis=1) / \
    total_df.groupby(level=[0], axis=1).sum()
score.fillna(0, inplace=True)

In [7]:
# 論文裡最高分的
nis_s_nis = score.rank(axis=1).loc['nis_s', :] - score.rank(axis=1).loc['nis', :]
short = nis_s_nis.sort_values()[:5]
long = nis_s_nis.sort_values()[-5:]
print(f'要Long這五檔: {long.index}')
print(f'要Short這五檔: {short.index}')

要Long這五檔: Index(['2498', '8069', '5425', '5483', '3034'], dtype='object')
要Short這五檔: Index(['2027', '2618', '2882', '2888', '2610'], dtype='object')


In [67]:
# 分成很多類來比較損益看看
# 1. 

# 損益
rt = np.log(stk_return.groupby('code')[['code', 'close']].tail(1).sort_values('code').set_index('code').astype(float) / \
        stk_return.groupby('code')[['code', 'close']].head(1).sort_values('code').set_index('code').astype(float))
coitype = score.index
for c in coitype:
        for c2 in coitype:
                if c == c2:
                        continue
                rank = score.rank(axis=1).loc[c, :] - score.rank(axis=1).loc[c2, :]
                long = rank.sort_values()[-5:]
                short = rank.sort_values()[:5]
                print('coitype: {} - {}, return: {:.4f}'.format(c, c2, (rt.loc[long.index].sum() - rt.loc[short.index].sum())["close"]))
                
                print('\n')
        for c2 in coitype:       
                rank = score.rank(axis=1).loc[c, :] + score.rank(axis=1).loc[c2, :]
                long = rank.sort_values()[-5:]
                short = rank.sort_values()[:5]
                print('coitype: {} + {}, return: {:.4f}'.format(c, c2, (rt.loc[long.index].sum() - rt.loc[short.index].sum())["close"]))        

coitype: all, return: 0.0827
coitype: iso, return: 0.0827
coitype: nis, return: 0.1399
coitype: nis_s, return: 0.1140
coitype: nis_c, return: -0.0320
coitype: nis_b, return: 0.0459


In [70]:
# 分成很多類來比較損益看看
# 1. 

# 損益
rt = np.log(stk_return.groupby('code')[['code', 'close']].tail(1).sort_values('code').set_index('code').astype(float) / \
        stk_return.groupby('code')[['code', 'close']].head(1).sort_values('code').set_index('code').astype(float))
coitype = score.index
for c in coitype:
    rank = score.rank(axis=1).loc[c, :]
    long = rank.sort_values()[-5:]
    short = rank.sort_values()[:5]
    print('coitype: {}, return: {:.4f}'.format(c, (rt.loc[long.index].sum() - rt.loc[short.index].sum())["close"]))
    for c2 in coitype:
        if c == c2:
                continue
        rank = score.rank(axis=1).loc[c, :] - score.rank(axis=1).loc[c2, :]
        long = rank.sort_values()[-5:]
        short = rank.sort_values()[:5]
        print('coitype: {} - {}, return: {:.4f}'.format(c, c2, (rt.loc[long.index].sum() - rt.loc[short.index].sum())["close"]))
            
    print('\n')
    for c2 in coitype:
        if c == c2:
                continue
        rank = score.rank(axis=1).loc[c, :] + score.rank(axis=1).loc[c2, :]
        long = rank.sort_values()[-5:]
        short = rank.sort_values()[:5]
        print('coitype: {} + {}, return: {:.4f}'.format(c, c2, (rt.loc[long.index].sum() - rt.loc[short.index].sum())["close"]))
    print('\n')

coitype: all, return: 0.0827
coitype: all - iso, return: 0.0579
coitype: all - nis, return: -0.0400
coitype: all - nis_s, return: -0.0556
coitype: all - nis_c, return: 0.1022
coitype: all - nis_b, return: 0.0368


coitype: all + iso, return: 0.0827
coitype: all + nis, return: 0.0907
coitype: all + nis_s, return: 0.1339
coitype: all + nis_c, return: 0.0133
coitype: all + nis_b, return: -0.0212


coitype: iso, return: 0.0827
coitype: iso - all, return: -0.0513
coitype: iso - nis, return: -0.0400
coitype: iso - nis_s, return: -0.0556
coitype: iso - nis_c, return: 0.1022
coitype: iso - nis_b, return: 0.0368


coitype: iso + all, return: 0.0827
coitype: iso + nis, return: 0.0907
coitype: iso + nis_s, return: 0.1339
coitype: iso + nis_c, return: 0.0133
coitype: iso + nis_b, return: -0.0212


coitype: nis, return: 0.1399
coitype: nis - all, return: 0.0400
coitype: nis - iso, return: 0.0400
coitype: nis - nis_s, return: -0.0477
coitype: nis - nis_c, return: -0.2084
coitype: nis - nis_b, return

In [50]:
rt.loc[long.index].sum() - rt.loc[short.index].sum()

close    0.082676
dtype: float64